<center><b> 1D time-dependent Frantz Nodvik equations </b></center>

<br> du1/dt + gamma*K*u1*u2 = 0           u1 = Delta = # density of excited states 
<br> du2/dt + c*du2/dx - K*u1*u2 = 0      u2 = n_ph  = # density of photons
<br>                                      c = speed of light
<br>                                      K = sigma * c
<br> IC:  TBD...
<br> BC:  TBD...
<br> This code is distributed under the Apache License, Version 2.0, January 2004.
<br> Copyright (c) 2020 RadiaSoft LLC, All rights reserved
<br> Author:  David Bruhwiler                                     
<br> The laser pulse duration $T_{laser}$ is typically 200 ps to 1 ns.
<br> The Ti-Sapphire phase relaxation time $T_{crystal}$ is between $10 fs$ and $10 ps$.
<br> Because $T_{crystal} << T_{laser}$, we can use the Frantz-Nodvik (F-N) rate equations.
<br> This approach can be used for other crystals as well.

In [1]:
# Primary library for the FEniCS Python API (should not import *)
from dolfin import *
from pykern import pkcollections
import scipy.constants

In [2]:
def frantz_nodvik_1d_solver(fn1d):
    """Frantz-Nodvik 1D solver for wavefront amplification in a pumped crystal amplifier

    Args:
        fn1d (PKDict): derived class of Python dict

    Returns:
        NA
    """
    
    # define the variational problem via function F
    F = ( (fn1d.u_1 - fn1d.u_n1) * fn1d.v_1 / fn1d.dt \
        + fn1d.c_gamma * fn1d.K * fn1d.u_1 * fn1d.u_2 * fn1d.v_1 \
        + (fn1d.u_2 - fn1d.u_n2) * fn1d.v_2 / fn1d.dt \
        + fn1d.c * fn1d.u_2.dx(0) * fn1d.v_2 \
        - fn1d.K * fn1d.u_1 * fn1d.u_2 * fn1d.v_2 \
        ) * dx

    # solve the system of equations
    solve(F==0, fn1d.u)
    
    return

In [3]:
def frantz_nodvik_1d_setup():
    """Setup choices for Frantz-Nodvik 1D solver

    Args:
        NA

    Returns:
        fn1d (object):  instance of the DataFrantzNodvik1D class
    """
    # specify the desired parameters (edit by hand for now)
    # create a Pykern Dict object to hold the data
    fn1d = pkcollections.PKDict()
    
    fn1d.sim_time = 1.       # [s]
    fn1d.n_steps = 1000
    fn1d.dt = fn1d.sim_time  \
            / fn1d.n_steps      # [s]
    fn1d.n_dx = 2000
    fn1d.c_gamma = 1.5          # dimensionless
    fn1d.c_sigma = 1.        # [m^2]
    fn1d.c_width = 0.01         # [m]
    fn1d.c_density = 1.      # [m^-3]
    fn1d.lp_density = 1       # [m^-3]
    fn1d.lp_width = 1.       # [s]
    
#    fn1d.c = scipy.constants.c         # speed of light [m/s]
    fn1d.c = 1.
    fn1d.K = fn1d.c_sigma * fn1d.c     # [m^3/s]
    
    # create a 1D mesh on the interval [0.,c_width].
    fn1d.x_left = 0.
    fn1d.x_right = fn1d.c_width
    fn1d.mesh = IntervalMesh(fn1d.n_dx, fn1d.x_left, fn1d.x_right)

    # let's try continuous line segments
    fn1d.p1 = FiniteElement("Lagrange","interval",1)
    
    # define an 'element' appropriate for two 1D functions
    fn1d.element = MixedElement([fn1d.p1, fn1d.p1])
    
    # define the test functions: v=[v_1, v_2]
    fn1d.v = FunctionSpace(fn1d.mesh, fn1d.element)
    fn1d.v_1, fn1d.v_2 = TestFunctions(fn1d.v)

    # define solution functions: u=[u_1, u_2]
    fn1d.u = Function(fn1d.v)
    fn1d.u_1, fn1d.u_2 = split(fn1d.u)

    # solutions from the previous time step: u_n=[u_n1, u_n2]
    fn1d.u_n = Function(fn1d.v)
    fn1d.u_n1, fn1d.u_n2 = split(fn1d.u_n)
    
    # specify and apply the initial conditions
    fn1d.v_1 = Constant(fn1d.c_density)  # assuming density of states is constant
    fn1d.v_2 = Constant(0.)              # assuming no photons in crystal at t=0

    # set U and U0 by interpolation.
    fn1d.u_n.assign(fn1d.u)

    return fn1d

In [4]:
# initialize the data object
fn1d = frantz_nodvik_1d_setup()

In [5]:
# solve the equations for one time step
frantz_nodvik_1d_solver(fn1d)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to define nonlinear variational problem F(u; v) = 0 for all v.
*** Reason:  Expecting the residual F to be a linear form (not rank 0).
*** Where:   This error was encountered inside NonlinearVariationalProblem.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.1.0
*** Git changeset:  74d7efe1e84d65e9433fd96c50f1d278fa3e3f3f
*** -------------------------------------------------------------------------
